# Criação do arquivo de impressão (.IoT)

## Bibliotecas

In [1]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
from enum import Enum, auto


## Variáveis

In [2]:
samplesImagesPath = os.listdir("../assets")
samplesImagesPath = map(
	lambda file: f"../assets/{file}", 
	samplesImagesPath
)
samplesImagesPath = list(samplesImagesPath)

## Função de extração de contornos

In [3]:
def getContours(image):
	contours = []

	cannedImage = cv.Canny(
		image,
		threshold1 = 100,
		threshold2 = 400
	)

	rawContours, _ = cv.findContours(
		cannedImage, 
		cv.RETR_LIST, 
		cv.CHAIN_APPROX_SIMPLE
	)

	for contour in rawContours:
		points = []

		for point in contour:
			x, y = point[0]

			points.append((x, y))

		contours.append(points)

	return contours

## Criação do arquivo .iot

In [4]:
image = cv.imread(
	samplesImagesPath[0], 
	cv.IMREAD_GRAYSCALE
)

contours = getContours(image)

with open("flor.iot", "w") as file:
	for contour in contours:

		firstPoint = contour[0]
		x0, y0 = firstPoint

		file.write("PENUP\n")
		file.write(f"\tGOTO {x0} {y0}\n")
		file.write("PENDOWN\n")

		for x, y in contour:
			file.write(f"\tGOTO {x} {y}\n")

## Enumeração dos comandos

In [5]:
class Commands(Enum):
	PENUP = auto()
	PENDOWN = auto()
	GOTO = auto()

## Interface para o Executor de comandos de impressão

In [6]:
class ExecutorInterface:
	def PENUP(self):
		raise NotImplementedError("PENUP method not implemented")
	
	def PENDOWN(self):
		raise NotImplementedError("PENDOWN method not implemented")
	
	def GOTO(self, x: int, y: int):
		raise NotImplementedError("GOTO method not implemented")

## Leitor do arquivo de impressão

In [7]:
class IotReader:
	def __init__(self, executor: ExecutorInterface):
		self.executor = executor

	def processCommand(self, command: Commands, args: list[int]):
		if command == Commands.PENUP:
			self.executor.PENUP()

		if command == Commands.PENDOWN:
			self.executor.PENDOWN()
		
		if command == Commands.GOTO:
			x, y = args
			self.executor.GOTO(x, y)

	def parseArgs(self, args: str) -> list[int]:
		return [int(arg) for arg in args.split()]
	
	def readLine(self, line: str):
		line = line.strip()

		args = []
		
		if " " in line:
			commandStr, argsStr = line.split(
				maxsplit = 1
			)

			self.processCommand(
				Commands[commandStr], 
				self.parseArgs(argsStr)
			)

			return			

		self.processCommand(Commands[line], args)

## Teste da leitura do arquivo de impressão com o executor Echo

In [ ]:
class EchoExecutor(ExecutorInterface):
	def PENUP(self):
		print("PENUP")
	
	def PENDOWN(self):
		print("PENDOWN")

	def GOTO(self, x, y):
		print(f"GOTO {x} {y}")
	
echoIotReader = IotReader(EchoExecutor()) 

with open("flor.iot", "r") as file:
	echoIotReader.readLine(file.readline())
	echoIotReader.readLine(file.readline())
	echoIotReader.readLine(file.readline())
	echoIotReader.readLine(file.readline())
	echoIotReader.readLine(file.readline())
	